In [60]:
import pandas as pd
import pickle

In [61]:
df_products = pd.read_json('../../../data/interim/final/products.json.gz', orient="records", compression="gzip")
df_reviews = pd.read_json('../../../data/interim/final/reviews.json.gz', orient="records", compression="gzip")

In [62]:
item_similarity_matrix = pickle.load(open(f'../../../models/content_based_filtering/item_similarity_matrix.pkl', 'rb'))

In [63]:
def find_similar_items(product_id, df_products):
    idx = df_products[df_products['product_id'] == product_id].index[0]
    sorted_scores = pd.Series(item_similarity_matrix[idx]).sort_values(ascending = False)
    return df_products.iloc[list(sorted_scores.iloc[1:].index)]

# Get unrated items for a user
def get_user_unrated_items(user_id, df_products, df_reviews):
    df_user_reviews = df_reviews[df_reviews['user_id'] == user_id]
    return df_products[~df_products['product_id'].isin(df_user_reviews['product_id'])]

def recommend(user_id, product_id, n = 10):
    df_similar_items = find_similar_items(product_id, df_products)
    df_user_unrated_items = get_user_unrated_items(user_id, df_similar_items, df_reviews)
    
    if n < len(df_user_unrated_items): 
        df_user_unrated_items = df_user_unrated_items[:n]
    
    return df_user_unrated_items

In [64]:
user_id = 'A0203183BAH3TR08FZGB'
product_id = 'B0043T7FHK'

In [65]:
df_products[df_products['product_id'] == product_id][['product_id', 'name', 'description']].head()

,product_id,name,description
226,B0043T7FHK,ASUS VE248H 24&quot; Full HD 1920x1080 2ms HDM...,The exclusive Splendid Video Intelligence Tech...


In [66]:
recommend(user_id, product_id, 10)[['product_id', 'name', 'description']]

,product_id,name,description
229,B0045JFTXU,ASUS VE208T 20&quot; HD+ 1600x900 DVI VGA Back...,Dynamically enhances the displays contrast by ...
393,B00A6AMYG2,Asus VK228H-CSM 21.5&quot; Widescreen LED Moni...,<b>True-to-life Pictures Powered by LED<br> 80...
408,B00ANKMNXO,ASUS VX248H 24&quot; Full HD 1920x1080 1ms HDM...,Superior Image Quality Meets Ultra-Slim Elegan...
240,B004G7U5LC,ASUS VW199T-P 19&quot; WXGA+ 1440x900 DVI VGA ...,Shipping Depth: 19.5 Shipping Height: 5.6 Ship...
95,B000VZG0QM,Asus VB171T - 17&quot; (4:3) LCD Monitors - Black,ASUS VB171T - with SPLENDID Video Intelligence...
267,B0058UUR6E,ASUS VS248H-P 24&quot; Full HD 1920x1080 2ms H...,Superior Image Quality Meets Classic Elegant D...
227,B0043T7FKC,ASUS VK278Q 27&quot; Full HD 1920x1080 2ms HDM...,Full HD with HDMI:VK278Q leads you to enter a ...
1451,B019HN6P36,ASUS VP247H-P 23.6 in LED Monitor 1920x1080 1m...,"With a 1 ms response time, the VP247H-P 23.6"" ..."
277,B005G2P16A,ASUS VS208N-P 20&quot; HD+ 1600x900 DVI VGA Ba...,20IN LCD 1600X900 16:9 LED built in power adap...
1341,B0163JMDLA,ASUS PRO C424AQ 23.8&rdquo; Full HD 1920x1080 ...,"Designed for business versatility, the all-new..."
